In [78]:
from lusidtools.jupyter_tools import toggle_code

"""
Save externally calculated metrics and use them within the Valuation Engine

Attributes
----------
"""

toggle_code("Toggle Docstring")

# Externally calculated metrics 

In [107]:
# Use first block to import generic non-LUSID packages
import io
import os
import pandas as pd
import numpy as np
import json
import pytz
from IPython.core.display import HTML
from datetime import datetime

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.7715.0,0.5.2373,"{'relation': 'RequestLogs', 'href': 'http://de..."


# Setup

In [108]:
scope = "srs-example"

## Instruments

In [109]:
instruments_df = pd.DataFrame(
    data=[
        ["BARC LN", "Barclays"],
        ["TSCO LN", "Tesco"],
        ["VOD LN", "Vodafone Group"]
    ],
    columns=["ClientInternal", "Name"]    
)

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instruments_df,
    mapping_required={
        "name": "Name"        
    },
    mapping_optional={},
    file_type="instruments",
    identifier_mapping={
        "ClientInternal": "ClientInternal"
    }
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,3,0,0


## Properties

In [110]:
def create_shk(shk_code, display_name):
    try:
        api_factory.build(lu.PropertyDefinitionsApi).create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="Transaction",
                scope=scope,
                code=shk_code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code="string"),
                life_time=None,
            )
        )
    except lu.ApiException as e:
        display(json.loads(e.body)["title"])

In [111]:
create_shk("Strategy", "Strategy")
create_shk("Sector", "Sector")
create_shk("WatchList", "WatchList")

"Error creating Property Definition 'Transaction/srs-example/Strategy' because it already exists."

"Error creating Property Definition 'Transaction/srs-example/Sector' because it already exists."

"Error creating Property Definition 'Transaction/srs-example/WatchList' because it already exists."

## Portfolio

In [112]:
portfolio_df = pd.DataFrame(
    data=[
        ["UKEquityA", "UK Equity Fund A", "GBP"],
        ["UKEquityB", "UK Equity Fund B", "GBP"]
    ],
    columns=["code", "display_name", "base_currency"]
)

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolio_df,
    mapping_required={
        "code": "code",
        "display_name": "display_name",
        "base_currency": "base_currency"
    },
    mapping_optional={
        "created": "$2020-01-01T00:00:00+00:00"
    },
    file_type="portfolios",
    sub_holding_keys=["Strategy", "Sector", "WatchList"],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,2,0,0


## Holdings

In [113]:
day1 = datetime(2021, 8, 1, tzinfo=pytz.utc)
day2 = datetime(2021, 8, 2, tzinfo=pytz.utc)
day3 = datetime(2021, 8, 3, tzinfo=pytz.utc)

holdings_df = pd.DataFrame(
    data=[
        [day1, day1, "UKEquityA", "BARC LN", 2000, 3200, 1.6, 3600, 400, "Growth", "Financial", "N", "TAXLOT001", "GBP"],
#         [day1, day1, "UKEquityA", "BARC LN", 3000, 5100, 1.7, 5400, 300, "Growth", "Financial", "TAXLOT002", "GBP"],
        [day1, day1, "UKEquityA", "TSCO LN", 3000, 6600, 2.2, 7200, 600, "Value", "Consumer", "Y", "TAXLOT003", "GBP"],
        [day1, day1, "UKEquityA", "BARC LN", 3500, 5600, 1.6, 6300, 700, "Value", "Financial", "N", "TAXLOT005", "GBP"],
        [day1, day1, "UKEquityB", "BARC LN", 5000, 9500, 1.9, 9000, -500, "Growth", "Financial", "N", "TAXLOT004", "GBP"],
        [day1, day1, "UKEquityB", "VOD LN", 2000, 3400, 1.7, None, None, "Growth", "Financial", "N", "TAXLOT004", "GBP"],
        
        [day1, day2, "UKEquityA", "BARC LN", 2000, 3200, 1.6, 3700, 500, "Growth", "Financial", "N", "TAXLOT001", "GBP"],
#         [day1, day2, "UKEquityA", "BARC LN", 3000, 5100, 1.7, 5550, 450, "Growth", "Financial", "TAXLOT002", "GBP"],
        [day1, day2, "UKEquityA", "TSCO LN", 3000, 6600, 2.2, 7500, 900, "Value", "Consumer", "Y", "TAXLOT003", "GBP"],
        [day1, day2, "UKEquityA", "BARC LN", 3500, 5600, 1.6, 6475, 875, "Value", "Financial", "N", "TAXLOT005", "GBP"],
        [day1, day2, "UKEquityB", "BARC LN", 5000, 9500, 1.9, 9250, -250, "Growth", "Financial", "N", "TAXLOT004", "GBP"],

    ],
    columns=["PurchaseDate", "ValuationDate", "Portfolio", "ClientInternal", "Units", 
             "Cost", "Price", "MV", "GainLoss", "Strategy", "Sector", "WatchList", "Taxlot", "Currency"]    
)

holdings_df

,PurchaseDate,ValuationDate,Portfolio,ClientInternal,Units,Cost,Price,MV,GainLoss,Strategy,Sector,WatchList,Taxlot,Currency
0,2021-08-01 00:00:00+00:00,2021-08-01 00:00:00+00:00,UKEquityA,BARC LN,2000,3200,1.60,3600.00,400.00,Growth,Financial,N,TAXLOT001,GBP
1,2021-08-01 00:00:00+00:00,2021-08-01 00:00:00+00:00,UKEquityA,TSCO LN,3000,6600,2.20,7200.00,600.00,Value,Consumer,Y,TAXLOT003,GBP
2,2021-08-01 00:00:00+00:00,2021-08-01 00:00:00+00:00,UKEquityA,BARC LN,3500,5600,1.60,6300.00,700.00,Value,Financial,N,TAXLOT005,GBP
3,2021-08-01 00:00:00+00:00,2021-08-01 00:00:00+00:00,UKEquityB,BARC LN,5000,9500,1.90,9000.00,-500.00,Growth,Financial,N,TAXLOT004,GBP
4,2021-08-01 00:00:00+00:00,2021-08-01 00:00:00+00:00,UKEquityB,VOD LN,2000,3400,1.70,nan,nan,Growth,Financial,N,TAXLOT004,GBP
5,2021-08-01 00:00:00+00:00,2021-08-02 00:00:00+00:00,UKEquityA,BARC LN,2000,3200,1.60,3700.00,500.00,Growth,Financial,N,TAXLOT001,GBP
6,2021-08-01 00:00:00+00:00,2021-08-02 00:00:00+00:00,UKEquityA,TSCO LN,3000,6600,2.20,7500.00,900.00,Value,Consumer,Y,TAXLOT003,GBP
7,2021-08-01 00:00:00+00:00,2021-08-02 00:00:00+00:00,UKEquityA,BARC LN,3500,5600,1.60,6475.00,875.00,Value,Financial,N,TAXLOT005,GBP
8,2021-08-01 00:00:00+00:00,2021-08-02 00:00:00+00:00,UKEquityB,BARC LN,5000,9500,1.90,9250.00,-250.00,Growth,Financial,N,TAXLOT004,GBP


In [114]:
holdings_api = api_factory.build(lu.TransactionPortfoliosApi)

In [115]:
portfolios = holdings_df.groupby(["Portfolio", "ValuationDate"])

for pf, pf_df in portfolios:
    
    tax_lot_hdlgs = pf_df.groupby(["ClientInternal", "Strategy", "Sector", "WatchList"])

    holdings_request = []    
    
    for h, tax_lots_df in tax_lot_hdlgs:
        
        tax_lots = []
        properties = {}

        for _, h in tax_lots_df.iterrows():

            tax_lots.append(
                lm.TargetTaxLotRequest(
                    units=h["Units"],
                    cost=lm.CurrencyAndAmount(amount=h["Cost"], currency=h["Currency"]),
                    price=h["Price"],
                    purchase_date=h["PurchaseDate"]
                )
            )

            properties[f"Transaction/{scope}/Strategy"] = lm.PerpetualProperty(
                key=f"Transaction/{scope}/Strategy", value=lm.PropertyValue(label_value=h["Strategy"])
            )

            properties[f"Transaction/{scope}/Sector"] = lm.PerpetualProperty(
                key=f"Transaction/{scope}/Sector", value=lm.PropertyValue(label_value=h["Sector"])
            )
            
            if h["WatchList"] == "Y":
                properties[f"Transaction/{scope}/WatchList"] = lm.PerpetualProperty(
                    key=f"Transaction/{scope}/WatchList", value=lm.PropertyValue(label_value=h["WatchList"])
                )

        holdings_request.append(lm.AdjustHoldingRequest(
            instrument_identifiers={ "Instrument/default/ClientInternal": h["ClientInternal"] },
            tax_lots=tax_lots,
            sub_holding_keys=properties
        ))
        
    holdings_api.set_holdings(scope=scope, code=pf[0], effective_at=h["ValuationDate"], adjust_holding_request=holdings_request)


# Structured Results Store

In [116]:
srs_api = api_factory.build(lu.StructuredResultDataApi)
instruments_api = api_factory.build(lu.InstrumentsApi)

In [117]:
srs_source_df = holdings_df[["ValuationDate", "Portfolio", "Strategy", "Sector", "WatchList", "ClientInternal", "Currency", "MV", "GainLoss"]]

In [118]:
instruments = instruments_api.get_instruments(identifier_type="ClientInternal", request_body=list(srs_source_df["ClientInternal"].unique()))

In [119]:
ci_to_luid = {
    ci: inst.identifiers["LusidInstrumentId"]
    for ci, inst in instruments.values.items()
}

srs_source_df["Scope"] = scope
srs_source_df["Luid"] = srs_source_df.apply(lambda x: ci_to_luid[x["ClientInternal"]], axis=1)
display(srs_source_df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss,Scope,Luid
0,2021-08-01 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3600.00,400.00,srs-example,LUID_CIEWJLSS
1,2021-08-01 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7200.00,600.00,srs-example,LUID_WT2BB71B
2,2021-08-01 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6300.00,700.00,srs-example,LUID_CIEWJLSS
3,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9000.00,-500.00,srs-example,LUID_CIEWJLSS
4,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,VOD LN,GBP,nan,nan,srs-example,LUID_YBR7UN56
5,2021-08-02 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3700.00,500.00,srs-example,LUID_CIEWJLSS
6,2021-08-02 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7500.00,900.00,srs-example,LUID_WT2BB71B
7,2021-08-02 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6475.00,875.00,srs-example,LUID_CIEWJLSS
8,2021-08-02 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9250.00,-250.00,srs-example,LUID_CIEWJLSS


## Data Map

In [120]:
srs_data_map = lm.DataMapping(
    data_definitions=[

        # composite key         
        lm.DataDefinition(address="UnitResult/Portfolio/Id", name="Portfolio", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address="UnitResult/Portfolio/Scope", name="Scope", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address=f"UnitResult/Transaction/{scope}/Strategy", name="Strategy", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address=f"UnitResult/Transaction/{scope}/Sector", name="Sector", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address=f"UnitResult/Transaction/{scope}/WatchList", name="WatchList", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address="UnitResult/Instrument/default/LusidInstrumentId", name="Luid", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address="UnitResult/Holding/default/Currency", name="Currency", data_type="string", key_type="PartOfUnique"),

        # holding values         
        lm.DataDefinition(address="UnitResult/MV", name="MV", data_type="decimal", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/GainLoss", name="GainLoss", data_type="decimal", key_type="Leaf"),
    ]
)

srs_data_map_key = lm.DataMapKey(version="0.1.4", code="market-valuation-map")

# display(srs_data_map)

try:    
    srs_api.create_data_map(
        scope=scope, 
        request_body={
            "market-valuation-map": lm.CreateDataMapRequest(
                id=srs_data_map_key,
                data=srs_data_map
            )
        }
    )
except lu.ApiException as e:
    display(json.loads(e.body))

{'name': 'StandardResourceConflict',
 'errorDetails': [],
 'code': 461,
 'type': 'https://docs.lusid.com/#section/Error-Codes/461',
 'title': 'Could not validate command. Reason given: DataMap exists',
 'status': 404,
 'detail': 'Could not validate command. Reason given: DataMap exists',
 'instance': 'https://demosetup.lusid.com/app/insights/logs/0HMBEO6DA27EP:00000003',
 'extensions': {}}

## Data

In [121]:
srs_ids = []

for effective_at, srs_df in srs_source_df.groupby("ValuationDate"):

    srs_data_id = lm.StructuredResultDataId(
        source="Client",
        code="MarketValuation",
        effective_at=effective_at,
        result_type = "UnitResult/Holding"
    )
    
    srs_ids.append(srs_data_id)
    
    # filter out rows without any MV and GainLoss values
    srs_df = srs_df[~srs_df[["MV", "GainLoss"]].isna().all(1)]
    
    s = io.StringIO()
    srs_df.to_csv(s)
    
    srs_data = lm.StructuredResultData(
        document_format="Csv",
        version="0.1.1",
        name="Market valuations",
        data_map_key=srs_data_map_key,
        document=s.getvalue()        
    )
    
    srs_api.upsert_structured_result_data(
        scope=scope, 
        request_body={ 
            "data": lm.UpsertStructuredResultDataRequest(
                id=srs_data_id, 
                data=srs_data
            )
        }
    )

## Read from SRS

In [122]:
for sid in srs_ids:
    
    key = f"{sid.code}-{effective_at}"
    
    values = srs_api.get_structured_result_data(
        scope=scope, 
        request_body={
            key: sid
        }
    )
    
    s = io.StringIO(values.values[key].document)
    values_df = pd.read_csv(s)
    
    display(values_df)

,Unnamed: 0,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss,Scope,Luid
0,0,2021-08-01 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3600.00,400.00,srs-example,LUID_CIEWJLSS
1,1,2021-08-01 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7200.00,600.00,srs-example,LUID_WT2BB71B
2,2,2021-08-01 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6300.00,700.00,srs-example,LUID_CIEWJLSS
3,3,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9000.00,-500.00,srs-example,LUID_CIEWJLSS


,Unnamed: 0,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss,Scope,Luid
0,5,2021-08-02 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3700.00,500.00,srs-example,LUID_CIEWJLSS
1,6,2021-08-02 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7500.00,900.00,srs-example,LUID_WT2BB71B
2,7,2021-08-02 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6475.00,875.00,srs-example,LUID_CIEWJLSS
3,8,2021-08-02 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9250.00,-250.00,srs-example,LUID_CIEWJLSS


# Valuation Engine

## Recipe

In [123]:
recipe_code = "MarketValuation"

recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    pricing=lm.PricingContext(
        result_data_rules=[
            lm.ResultDataKeyRule(
                resource_key="UnitResult/*",
                supplier="Client",
                data_scope=scope,
                document_code="MarketValuation",
                quote_interval="0D"
            )
        ],
        options = lm.PricingOptions(
            allow_partially_successful_evaluation=True,
            allow_any_instruments_with_sec_uid_to_price_off_lookup=False
        )                              
    )
)

configuration_recipe_api = api_factory.build(lu.ConfigurationRecipeApi)
upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request = lm.UpsertRecipeRequest(
        configuration_recipe = recipe
    )
)

In [124]:
aggregation_api = api_factory.build(lu.AggregationApi)

def run_valuation(portfolio_code, effective_at):
    
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code=recipe_code),
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=effective_at.isoformat()),
        metrics=[
            lm.AggregateSpec("Portfolio/Scope", "Value"),
            lm.AggregateSpec("Portfolio/Id", "Value"),
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec(f"Transaction/{scope}/Strategy", "Value"),
            lm.AggregateSpec(f"Transaction/{scope}/Sector", "Value"),
            lm.AggregateSpec(f"Transaction/{scope}/WatchList", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("UnitResult/MV", "Value"),
            lm.AggregateSpec("UnitResult/GainLoss", "Value"),
        ]
    )
    
    valuation_result = aggregation_api.get_valuation(valuation_request=valuation_request)
    
    display(pd.DataFrame(valuation_result.data))

In [125]:
run_valuation("UKEquityA", day1)
run_valuation("UKEquityB", day1)
run_valuation("UKEquityA", day2)
run_valuation("UKEquityB", day2)

,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss
0,srs-example,UKEquityA,Barclays,Growth,Financial,<Not Classified>,2000.00,3600.00,400.00
1,srs-example,UKEquityA,Barclays,Value,Financial,<Not Classified>,3500.00,6300.00,700.00
2,srs-example,UKEquityA,Tesco,Value,Consumer,Y,3000.00,7200.00,600.00


,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss,Aggregation/Errors
0,srs-example,UKEquityB,Barclays,Growth,Financial,<Not Classified>,5000.00,9000.00,-500.00,[]
1,srs-example,UKEquityB,Vodafone Group,Growth,Financial,<Not Classified>,2000.00,nan,nan,[Failed to resolve result data item [UnitResul...


,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss
0,srs-example,UKEquityA,Barclays,Growth,Financial,<Not Classified>,2000.00,3700.00,500.00
1,srs-example,UKEquityA,Barclays,Value,Financial,<Not Classified>,3500.00,6475.00,875.00
2,srs-example,UKEquityA,Tesco,Value,Consumer,Y,3000.00,7500.00,900.00


,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss
0,srs-example,UKEquityB,Barclays,Growth,Financial,<Not Classified>,5000.00,9250.00,-250.00


In [126]:
run_valuation("UKEquityB", day3)

,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss
0,srs-example,UKEquityB,Barclays,Growth,Financial,<Not Classified>,5000.00,9250.00,-250.00
